In [39]:
print('Hello World')

Hello World


In [40]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, desc, year,avg, round

In [41]:
spark = SparkSession.builder \
    .appName("AnaliseInterativaClima") \
    .master("spark://spark-master:7077") \
    .config("spark.sql.warehouse.dir", "/opt/bitnami/spark/spark-warehouse") \
    .enableHiveSupport() \
    .getOrCreate()

25/09/09 23:36:54 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [42]:
df_clima = spark.read.parquet(
            "data/input/clima_inmet_aggregated.parquet",
            header=True,
            inferSchema=True
        )

In [43]:
df_clima.show(5)
#df_clima.printSchema()

+----------+------+---+--------------------+---------------+---------------+------------------+
|      Data|Regiao| UF|              Cidade|Max_Temperatura|Min_Temperatura|   Med_Temperatura|
+----------+------+---+--------------------+---------------+---------------+------------------+
|2009-01-01|    CO| DF|          Planaltina|           28.8|           17.3|22.954166666666666|
|2009-01-01|    CO| DF|Sudoeste e Octogonal|           29.0|           19.3|23.229166666666668|
|2009-01-01|    CO| GO|Alto Paraíso de G...|           28.0|           17.2|21.941666666666666|
|2009-01-01|    CO| GO|           Aragarças|           30.4|           23.1|27.054166666666664|
|2009-01-01|    CO| GO|           Caiapônia|           31.1|           21.4|24.479166666666668|
+----------+------+---+--------------------+---------------+---------------+------------------+
only showing top 5 rows


In [8]:
df_clima.write.parquet(
        "/opt/bitnami/spark/data/output/clima_agregado",
        mode="overwrite"
    )

In [44]:
df_clima.createOrReplaceTempView("clima_view")

In [45]:
sql_query = """
SELECT DISTINCT Cidade
    FROM 
        clima_view
WHERE Cidade like '%Belo%'
"""

In [46]:
cidades = spark.sql(sql_query)

In [47]:
cidades.show()

[Stage 34:=============================>                            (1 + 1) / 2]

+--------------+
|        Cidade|
+--------------+
|Belo Horizonte|
+--------------+



In [17]:
sql_query = """
SELECT *
FROM clima_view
WHERE Cidade = 'Belo Horizonte'
"""

In [18]:
belo_horizonte = spark.sql(sql_query)

In [19]:
belo_horizonte.show()

+----------+------+---+--------------+---------------+---------------+------------------+
|      Data|Regiao| UF|        Cidade|Max_Temperatura|Min_Temperatura|   Med_Temperatura|
+----------+------+---+--------------+---------------+---------------+------------------+
|2009-01-01|    SE| MG|Belo Horizonte|           29.0|           16.6|21.702083333333334|
|2009-01-02|    SE| MG|Belo Horizonte|           28.7|           17.6|22.827083333333334|
|2009-01-03|    SE| MG|Belo Horizonte|           27.5|           18.2| 21.74791666666667|
|2009-01-04|    SE| MG|Belo Horizonte|           27.7|           17.6|20.945833333333333|
|2009-01-05|    SE| MG|Belo Horizonte|           22.1|           16.3| 18.84318181818182|
|2009-01-06|    SE| MG|Belo Horizonte|           27.5|           17.2| 21.56578947368421|
|2009-01-07|    SE| MG|Belo Horizonte|           28.7|           16.9| 21.96818181818182|
|2009-01-08|    SE| MG|Belo Horizonte|           28.9|           18.0| 22.74791666666667|
|2009-01-0

In [23]:
sql_query = """
SELECT YEAR(Data) as Ano
        ,MIN(Min_Temperatura) as Temperatura_Minima
        ,MAX(Max_Temperatura) as Temperatura_Maxima
FROM clima_view
WHERE Cidade = 'Belo Horizonte'
GROUP BY YEAR(Data)
Order by Ano
"""
belo_horizonte_report = spark.sql(sql_query)

In [26]:
belo_horizonte_report.show()

+----+------------------+------------------+
| Ano|Temperatura_Minima|Temperatura_Maxima|
+----+------------------+------------------+
|2009|               9.1|              33.0|
|2010|               6.6|              32.9|
|2011|               9.5|              33.3|
|2012|               8.6|              36.1|
|2013|               9.1|              35.9|
|2014|               8.4|              36.0|
|2015|               9.6|              37.0|
|2016|               9.0|              34.9|
|2017|               6.2|              35.3|
|2018|               8.3|              33.7|
|2019|               6.5|              35.5|
|2020|               8.1|              37.8|
|2021|               6.7|              36.1|
|2022|               4.5|              34.5|
|2023|               8.9|              37.6|
|2024|               9.5|              35.5|
+----+------------------+------------------+



In [27]:
sql_query = """
SELECT COUNT(*) as Qtd
FROM clima_view
"""
qtd_dados = spark.sql(sql_query)
qtd_dados.show()

+-------+
|    Qtd|
+-------+
|2649304|
+-------+



In [38]:
med_cidades = df_clima.groupBy("Cidade", "UF") \
    .agg(round(avg("Med_Temperatura"), 2).alias("Temperatura_Média")) \
    .orderBy(desc("Temperatura_Média"))

print("Cidades Brasileiras Temperaturas médias nos ultimos 15 anos:")
med_cidades.show()

Cidades Brasileiras Temperaturas médias nos ultimos 15 anos:
+--------------------+---+-----------------+
|              Cidade| UF|Temperatura_Média|
+--------------------+---+-----------------+
|Santo Antônio do Içá| AM|            30.59|
|           Ipanguaçu| RN|             29.3|
|           Jaguaribe| CE|            29.22|
|               Caicó| RN|            29.04|
|         Campo Maior| PI|            28.76|
|               Picos| PI|            28.75|
|   São João do Piauí| PI|            28.63|
|            Floriano| PI|            28.54|
|               Patos| PB|            28.45|
|São Miguel do Ara...| GO|            28.35|
|          Paulistana| PI|            28.22|
|              Cuiabá| MT|            28.13|
|            Piripiri| PI|            28.05|
|          Itaporanga| PB|            28.04|
|             Crateús| CE|            28.04|
|            Teresina| PI|            28.03|
|         Morada Nova| CE|            28.01|
|               Apodi| RN|             